In [1]:
import pandas as pd             # data package
import matplotlib.pyplot as plt # graphics 
import datetime as dt
import numpy as np
from census import Census # This is new...

import requests, io             # internet and input tools  
import zipfile as zf            # zip file tools 
import os  

#import weightedcalcs as wc
#import numpy as np

import pyarrow as pa
import pyarrow.parquet as pq

from bokeh.palettes import brewer, Spectral6
from bokeh.io import show, output_file, curdoc
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, HoverTool, Panel, Tabs, GeoJSONDataSource, LinearColorMapper
from bokeh.models import ColorBar
from bokeh.layouts import column, gridplot, row
from bokeh.transform import factor_cmap
from bokeh.models import NumeralTickFormatter, Title, Label, Paragraph, Div, CustomJSHover, BoxAnnotation


In [2]:
country_list = pd.read_csv('./data/top_50_non_eu.csv', dtype={'CTY_CODE': str})

country_list.loc[len(country_list)] = ["EUROPEAN UNION", 0 , "0003"]

country_list["tariff"] = 10.0

country_list.loc[country_list["CTY_CODE"] == "5700", "tariff"] = 30.0

country_list.loc[country_list["CTY_CODE"] == "1220", "tariff"] = 8.75

country_list.loc[country_list["CTY_CODE"] == "2010", "tariff"] = 4.5

country_list.loc[country_list["CTY_CODE"] == "0003", "tariff"] = 10.0
#IF EU you get 50 and you'll like it


In [3]:
country_list

,CTY_NAME,total_imports,CTY_CODE,tariff
0,CHINA,5.728638e+12,5700,30.00
1,MEXICO,4.385957e+12,2010,4.50
2,CANADA,4.161438e+12,1220,8.75
3,JAPAN,1.701399e+12,5880,10.00
4,"KOREA, SOUTH",1.045098e+12,5800,10.00
5,VIETNAM,8.800542e+11,5520,10.00
6,TAIWAN,7.522322e+11,5830,10.00
7,INDIA,7.405954e+11,5330,10.00
8,UNITED KINGDOM,7.096412e+11,4120,10.00
9,SWITZERLAND,6.056859e+11,4419,10.00


Step one is to read in the tariff lists, these come from the orginal Annex's published in the Federal Register

In [4]:
expemption_list = pd.read_csv('./tariff-lists/annex-II-exemptions.csv', dtype={'HTSUS': str})
# these are the annex II exemptions from the reciprical tariff list

expemption_list_phones = pd.read_csv('./tariff-lists/41225_exemptions.csv', dtype={'HTSUS': str})
# these are the phone exemptions

steel_list = pd.read_csv('./tariff-lists/steel-tariffs.csv', dtype={'HTSUS': str})
# these are the steel products subject to the new 232 tariffs. This is at 8 digit level.

alu_list = pd.read_csv('./tariff-lists/alu-tariffs.csv', dtype={'HTSUS': str})
# these are the aluminum products subject to the new 232 tariffs. This is at a mixed 8 and 10 digit level.

alu_8_list = alu_list[alu_list['HTSUS'].str.len() == 8]

auto_list = pd.read_csv('./tariff-lists/auto-tariffs.csv', dtype={'HTSUS': str})
# these are the aluminum products subject to the new 232 tariffs. This is at a mixed 8 and 10 digit level.

auto_6_list = auto_list[auto_list['HTSUS'].str.len() == 6]

auto_8_list = auto_list[auto_list['HTSUS'].str.len() == 8]


# country_list = pd.read_csv('./tariff-lists/reciprocal-tariffs-annex-I.csv',dtype={'CTY_CODE': str})

In [ ]:
def get_all_imports(cnty_code, tariff):

    my_key = "&key=34e40301bda77077e24c859c6c6c0b721ad73fc7"

    hs = 'HS10'

    my_key = "&key=34e40301bda77077e24c859c6c6c0b721ad73fc7"

    end_use = "hs?get=CTY_NAME,CON_VAL_YR,CAL_DUT_YR,I_COMMODITY,I_COMMODITY_SDESC"

    surl = "https://api.census.gov/data/timeseries/intltrade/imports/" + end_use 

    surl  = surl + my_key + "&time=" + "2024-12" + "&COMM_LVL=" + hs 

    url = surl + "&CTY_CODE=" + cnty_code

    r = requests.get(url) 
    
    print(r)
    
    df = pd.DataFrame(r.json()[1:]) # This then converts it to a dataframe
    # Note that the first entry is the labels

    df.columns = r.json()[0]

    #######################################################################

    df.time = pd.to_datetime(df.time, format="%Y-%m")

    df["imports"] = df["CON_VAL_YR"].astype(float)

    df["duty"] = df["CAL_DUT_YR"].astype(float)

    df["share"] = df["imports"] / df["imports"].sum()
            
    df[hs] = df["I_COMMODITY"].astype(str)

    df["HS8"] = df["HS10"].str[0:8]

    df["HS6"] = df["HS10"].str[0:6]

    #######################################################################
    # this then filters so products are only those that are subject to the new tariffs
    # the reciprocal exemptions, steel, aluminum and auto tariffs.

    df["tariff"] = tariff


    exemption = df["HS10"].isin(expemption_list["HTSUS"].tolist()) 

    df.loc[exemption, "tariff"]  = 0.0

    exempt_imports = df.loc[exemption, "imports"].sum()

    ####################################################
    # this is the phone exemption list. This is a 10 digit list.

    exemption_phones = df["HS10"].isin(expemption_list_phones["HTSUS"].tolist()) 

    if cnty_code == "5700": # if china then still Fentanyl 20 percent on the phones

        df.loc[exemption_phones, "tariff"] = 20.0

    else:

        df.loc[exemption_phones, "tariff"] = 0.0

    ####################################################

    steel = df["HS8"].isin(steel_list["HTSUS"].tolist())

    if cnty_code == "4120": # if the UK, then exempted from the increase

        df.loc[steel, "tariff"]  = 25.0 #25

    else:

        df.loc[steel, "tariff"]  = 50.0 #25


    steel_imports = df.loc[steel, "imports"].sum()

    ####################################################

    alu = df["HS10"].isin(alu_list["HTSUS"].tolist()) | df["HS8"].isin(alu_8_list["HTSUS"].tolist())

    if cnty_code == "4120":
       
        df.loc[alu, "tariff"]  = 25.0 

    else:

        df.loc[alu, "tariff"]  = 50.0
        
    alu_imports = df.loc[alu, "imports"].sum()

    ####################################################


    auto = df["HS8"].isin(auto_8_list["HTSUS"].tolist()) | df["HS10"].isin(auto_list["HTSUS"].tolist()) | df["HS6"].isin(auto_6_list["HTSUS"].tolist())

    if cnty_code != "1220" and cnty_code != "2010": # if canada or mexico, then assumption is no tariff

        df.loc[auto, "tariff"]  = 25.0

    auto_imports = df.loc[auto, "imports"].sum()

    ####################################################


    return df, exempt_imports, steel_imports, alu_imports, auto_imports

In [6]:
df = pd.DataFrame()

recipricol_summary = pd.DataFrame(columns=['country_name','total imports',"2024 tariff","effective tariff", "expempt imports", "steel imports", "alu imports", "auto imports"])

for index, row in country_list.iterrows():
    
    foo, exempt_imports, steel_imports, alu_import, auto_imports = get_all_imports(row['CTY_CODE'], row['tariff'])

    recipricol_summary.loc[index, 'country_name'] = foo['CTY_NAME'].iloc[0]
    
    recipricol_summary.loc[index, 'total imports'] = foo['imports'].sum()

    recipricol_summary.loc[index, '2024 tariff'] = 100*(foo["duty"].sum() / foo['imports'].sum())

    recipricol_summary.loc[index, 'effective tariff'] = ( foo['tariff'] * foo['share'] ).sum()

    recipricol_summary.loc[index, 'expempt imports'] = exempt_imports
    
    recipricol_summary.loc[index, 'steel imports'] = steel_imports
    
    recipricol_summary.loc[index, 'alu imports'] = alu_import
    
    recipricol_summary.loc[index, 'auto imports'] = auto_imports    


    df = df.append(foo)

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>


In [7]:
recipricol_summary.sort_values(by='total imports', ascending=False, inplace=True)

In [18]:
recipricol_summary.head(20)

,country_name,total imports,2024 tariff,effective tariff,expempt imports,steel imports,alu imports,auto imports
50,EUROPEAN UNION,597906784683.0,1.23103,12.165396,19189814054.0,4075410635.0,11809647077.0,69288585685.0
1,MEXICO,503806119292.0,0.250566,6.142319,8205315086.0,3622743519.0,21399351504.0,149979172522.0
0,CHINA,429753399481.0,10.884031,28.188161,4015373708.0,7419241338.0,17860573753.0,32699634151.0
2,CANADA,412555790905.0,0.101179,8.103414,90530905516.0,2654084614.0,10508257908.0,50990178340.0
3,JAPAN,151290260357.0,1.503807,16.242467,824272804.0,694811056.0,5180005371.0,55885071029.0
5,VIETNAM,141407132644.0,3.774985,8.147225,232904515.0,493795767.0,2746060850.0,3917910374.0
4,"KOREA, SOUTH",130471361727.0,0.21392,15.577822,3593028538.0,1502899745.0,5591276568.0,49491063359.0
6,TAIWAN,115284322157.0,0.93952,5.939464,472568240.0,2162535337.0,3661916147.0,3217101173.0
7,INDIA,87793999313.0,2.404218,10.502201,2102493500.0,1502742206.0,1556728755.0,2832788361.0
8,UNITED KINGDOM,68124315671.0,0.962949,13.577348,1285548459.0,349495364.0,1929538213.0,11956811526.0


In [9]:
recipricol_summary.to_csv("tariff-summary-06-03-2025.csv", index = False)

In [10]:
avg_tariff = ( recipricol_summary["effective tariff"]*recipricol_summary["total imports"] ).sum() / recipricol_summary["total imports"].sum() + 2.3 

print("Average Tariff Rate: ", avg_tariff)

Average Tariff Rate:  14.616612146912377


In [11]:
def make_empty_df():

    empty_df = pd.DataFrame(columns=['country_name','total imports',"2024 tariff","effective tariff", "expempt imports", "steel imports", "alu imports", "auto imports"])

    empty_df.loc[0, 'country_name'] = ''
    
    empty_df.loc[0, 'total imports'] = 0.0

    empty_df.loc[0, 'effective tariff'] = 0.0

    empty_df.loc[0, '2024 tariff'] = 0.0

    empty_df.loc[0, 'expempt imports'] = 0.0

    empty_df.loc[0, 'steel imports'] = 0.0

    empty_df.loc[0, 'alu imports'] = 0.0

    empty_df.loc[0, 'auto imports'] = 0.0

    return empty_df

In [12]:
def make_source(df):
    
    df["position"] = df.reset_index().index.values
        
    df["hover_label"] = (df["total imports"]/1000000000).map('{:,.1f}'.format)
        
    df["hover_label_2"] = (df["effective tariff"]).map('{:,.1f}'.format)
    
    df["hover_label_3"] = (df["2024 tariff"]).map('{:,.1f}'.format)

    source = ColumnDataSource(df)
    
    return source

In [13]:
def make_bar_chart(df):

    height = int(1.15*533)
    width = int(1.15*750)

    source = make_source(df)
        
    p = figure(plot_height=height, plot_width = width, title= "U.S. Tariff Increase Since Jan 20th by Country of Origin (Top 20 by Import Value)", x_range=df['country_name'],
           toolbar_location = 'below',
           tools = "reset")
            
    p.vbar(x = "country_name", top = "effective tariff", width = 0.6, alpha = 0.65,
       hatch_pattern = " ",hatch_alpha = 0.10, color = "red",
       source = source)
    
    y_custom = CustomJSHover(code=""" return '' + special_vars.data_y
            """)

##########################################################################
    TIMETOOLTIPS = """
    <div style="background-color:#F5F5F5; opacity: 0.95; border: 0px 0px 0px 0px">
        <div style = "text-align:left;">
            <span style="font-size: 13px; font-weight: bold">@country_name</span>
        </div>
        <div style = "text-align:left;">
            <span style="font-size: 13px; font-weight: bold">2024 Imports: $@hover_label Billion</span>
        </div>
        <div style = "text-align:left;">
            <span style="font-size: 13px; font-weight: bold">Applied Tariff Increase: @hover_label_2%</span>
        </div>
        <div style = "text-align:left;">
            <span style="font-size: 13px; font-weight: bold">2024 Applied Tariff: @hover_label_3%</span>
        </div>
    </div>
    """

    p.add_tools(HoverTool(tooltips = TIMETOOLTIPS))
##########################################################################

    df["position"] = df.reset_index().index.values
    mid_idx = len(df['position']) // 2
    x_val = df['position'].iloc[mid_idx]
    y_val = df["effective tariff"].max() * .85

    mytext = Label(x=x_val, y=y_val, text='U.S. Average Tariff:', 
               text_font_size="2em", text_font_style="bold",
               x_units='data', y_units='data', render_mode='canvas')

    p.add_layout(mytext)

    mytext2 = Label(x=x_val, y=y_val * 0.75, text=str(round(avg_tariff,1)) + '%',
                text_font_size="6em", text_font_style="bold",
                x_units='data', y_units='data', render_mode='canvas')

    p.add_layout(mytext2)

    #p.ygrid.grid_line_color = None
    p.xgrid.grid_line_color = None
    
    p.title.text_font_size = '14pt'
    p.xaxis.major_tick_line_color = None  # turn off x-axis major ticks
    p.xaxis.minor_tick_line_color = None  # turn off x-axis minor ticks


    p.xaxis.major_label_text_font_size = '7pt'  # turn off x-axis tick labels
    p.xaxis.major_label_orientation = 0.75 

    p.yaxis.formatter = NumeralTickFormatter(format="(0. a)")
    p.yaxis.minor_tick_line_color = None
    p.y_range.start = 0 
    
    p.y_range.end = df["effective tariff"].max() + 0.10*df["effective tariff"].max()
    
    p.border_fill_color = background    
    
    p.background_fill_color = background 
    p.background_fill_alpha = 0.75    
    
    p.toolbar.autohide = True
    
    p.outline_line_color = None
    p.sizing_mode= "scale_both"
    p.max_height = height
    p.max_width = width
    p.min_height = int(0.25*height)
    p.min_width = int(0.25*width)

    
    return p

In [14]:
crl = ["darkblue","slategray","slategray","crimson","crimson"]

background = "#ffffff"

In [15]:
fed_tax = pd.read_csv("federal-tax-duty.csv",skiprows=3, nrows=17)
# from Table 3.5. Taxes on Production and Imports

fed_duties = fed_tax.iloc[15,2:]

# Convert the Series to a DataFrame
fed_duties_df = fed_duties.to_frame()

# # Optionally, rename the column for clarity
fed_duties_df.columns = ['duty']

fed_duties_df.index.rename('time', inplace=True)

# # # Reset the index if needed
fed_duties_df.reset_index(inplace=True)

fed_duties_df["time"] = pd.to_datetime(fed_duties_df["time"], format="%Y")

# Extract only the year and overwrite the 'time' column
fed_duties_df["time"] = fed_duties_df["time"].dt.year

######################################################################################

NIPA_imports = pd.read_csv("NIPA-imports.csv",skiprows=3, nrows=20)
# from Table 4.1. Foreign Transactions in the National Income and Product Accounts

imports = NIPA_imports.iloc[19,2:]

imports_df = imports.to_frame()

# Optionally, rename the column for clarity
imports_df.columns = ['import_value']

# Rename the index
imports_df.index.rename('time', inplace=True)

# Reset the index if needed
imports_df.reset_index(inplace=True)

# Ensure the 'time' column is in datetime format
imports_df["time"] = pd.to_datetime(imports_df["time"], format="%Y")

# Extract only the year and overwrite the 'time' column
imports_df["time"] = imports_df["time"].dt.year

########################################################################################

merged_df = pd.merge(fed_duties_df, imports_df, on='time', how='inner')

merged_df["duty"] = merged_df["duty"].astype(float)
merged_df["import_value"] = merged_df["import_value"].astype(float)

merged_df["tariff"] = ( merged_df["duty"] / merged_df["import_value"] ) * 100

# Create a new row with the year 2024 and a tariff value
# comes from www.tradewartracker.com
new_row = {'time': "2024", 'duty': 0, 'import_value': 0, 'tariff': 2.4}

# Append the new row to the DataFrame
merged_df = merged_df.append(new_row, ignore_index=True)

# Create a new row with the year 2024 and a tariff value
# comes from www.tradewartracker.com
new_row = {'time': "2025", 'duty': 0, 'import_value': 0, 'tariff': avg_tariff}

# Append the new row to the DataFrame
merged_df = merged_df.append(new_row, ignore_index=True)

# Ensure the 'time' column is in datetime format
merged_df["time"] = pd.to_datetime(merged_df["time"], format="%Y")

# # Extract only the year and overwrite the 'time' column
# merged_df["time"] = merged_df["time"].dt.year

In [16]:
def make_tariff_time(df):

    height = int(1.15*533)
    width = int(1.15*750)

    source = ColumnDataSource(df)
        
    p = figure(x_axis_type="datetime", plot_height=height, plot_width = width, title= "U.S. Tariff Rate 1929-2025",
           toolbar_location = 'below',
           tools = "reset")
            
    p.line(x = "time", y = "tariff",
                line_width=7, line_alpha=0.75, line_color = "slategray",
                hover_line_alpha=0.75, hover_line_width = 7,
                hover_line_color= "crimson", source = source)
    
    # p.circle(
    #     x=pd.to_datetime("2025"),  # Specify the x-coordinate (time)
    #     y=22.5,                     # Specify the y-coordinate (tariff value)
    #     size=15,
    #     alpha = 0.75,            # Transparency of the circle
    #     color="red",               # Color of the circle
    # )
    
##########################################################################
    TIMETOOLTIPS = """
            <div style="background-color:#F5F5F5; opacity: 0.95; border: 5px 5px 5px 5px;">
            <div style = "text-align:left;">
            <span style="font-size: 13px; font-weight: bold"> Average Tariff
             </span>
             </div>
             <div style = "text-align:left;">"""
    

    TIMETOOLTIPS = TIMETOOLTIPS + """
            <span style="font-size: 13px; font-weight: bold"> $data_x{%Y}:  $data_y{0.0}%</span>   
            </div>
            </div>
            """
        
    p.add_tools(HoverTool(tooltips = TIMETOOLTIPS,  line_policy='nearest', formatters={'$data_x': 'datetime'}))

    # p.add_tools(HoverTool(tooltips = TIMETOOLTIPS))
##########################################################################

    p.xgrid.grid_line_color = None
    
    p.title.text_font_size = '14pt'

    p.xaxis.major_tick_line_color = None  # turn off x-axis major ticks
    p.xaxis.minor_tick_line_color = None  # turn off x-axis minor ticks

    p.x_range.start = pd.to_datetime("1928-01-01")
    p.xaxis.major_label_text_font_size = '12pt'  # turn off x-axis tick labels
    p.xaxis.major_label_orientation = 0.75 

    p.yaxis.formatter = NumeralTickFormatter(format="(0. a)")
    # p.yaxis.minor_tick_line_color = None
    p.y_range.start = 0 
    p.y_range.end = 30.0 

    p.yaxis.major_label_text_font_size = "12pt"  # Increase font size
    
    # p.y_range.end = df["effective tariff"].max() + 0.10*df["effective tariff"].max()
    
    p.border_fill_color = background    
    
    p.background_fill_color = background 
    p.background_fill_alpha = 0.75    
    
    p.toolbar.autohide = True
    
    p.outline_line_color = None
    p.sizing_mode= "scale_both"
    p.max_height = height
    p.max_width = width
    p.min_height = int(0.25*height)
    p.min_width = int(0.25*width)

    
    return p

In [17]:
foo = recipricol_summary[0:20].copy(deep=True)

foobar = make_empty_df()

foobar = foobar.append(foo)

p1 = make_bar_chart(foobar)

p2 = make_tariff_time(merged_df)

tab2 = Panel(child= p2, title="Tariffs Over Time")

output_file('.\\docs\\' + "reciprocal-new-steel.html")

div1 = Div(text = """Each bar represents the increase in the trade-weighted applied tariff for each country since January 20th. Tariffs include reciprocal tariffs, 
           steel and aluminum tariffs, auto tariffs, and the 5/12 US-China pause. Includes 4/2 and 4/12 (smartphone) exemptions. Fentanyl tariffs for Canada and Mexico are calculated under the assumption that all auto 
           imports are USMCA-compliant, and that 65% of Canadian imports and 82% of Mexican imports are USMCA-compliant. The average tariff 
           reflects the trade-weighted average of all tariffs, including the existing 2.3% applied tariff as of February 2025.
""", max_width= int(1.15*750), background = background )


div2 = Div(text = """This chart shows the average U.S. tariff rate from 1929 to the present.
The average tariff is calculated as collected duties (from NIPA Table 3.5) divided by the value of goods imports (from NIPA Table 4.1).
The 2025 estimated tariff rate for 2025 includes reciprocal tariffs, steel and aluminum tariffs, auto tariffs,
and the fentanyl-related tariffs. These estimates account for exemptions where applicable.   
""", max_width= int(1.15*750), background = background )

tab1_layout = column(p1, div1, sizing_mode="scale_both")
tab2_layout = column(p2, div2, sizing_mode="scale_both")

# Create tabs
tab1 = Panel(child=tab1_layout, title="Tariffs by Country")
tab2 = Panel(child=tab2_layout, title="Tariffs Over Time")

# Combine tabs into a Tabs layout
tabs = Tabs(tabs=[tab1, tab2], tabs_location="above")

show(tabs)